In [1]:
from src.constants import places
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os
import json

In [2]:
from pymongo import MongoClient

client = MongoClient()

def connectCollection(database, collection):
    ''' return collection from db crunchbase'''
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('dbcompanies','companies_clean')

In [4]:
companies = db.companies_clean.find({ 
    "money_raised":{
        "$gte": 1000000
    }}) 

In [5]:
data_companies = pd.DataFrame(companies)

In [6]:
geo_point = []
for loc in range(len(data_companies)):
    geo_point.append(data_companies.coordinates[loc])  

In [7]:
def near(geo_point, radio_meters):
        geopoint = geo_point
        return list(db.companies_clean.find({
        "coordinates.coordinates": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }}})) 

radio_meters = 3000
number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  near(geo_point[i], radio_meters)
    list_offices.append(num_offices)
    number_offices.append(len(num_offices))
data_companies['number offices near'] = number_offices

In [8]:
data_companies = data_companies[data_companies['number offices near'] > 1]

In [114]:
data_companies.head()

,_id,name,employees,year,category,id,money_raised,city,country,latitude,longitude,coordinates,number offices near
0,5e049e9de6eb90cad2d707ad,Fixya,30.0,2013,web,52cdef7c4bab8bd675297fec,8000000.0,San Mateo,USA,37.566879,-122.323895,"{'type': 'Point', 'coordinates': [-122.323895,...",2
2,5e049e9de6eb90cad2d707af,Social Gaming Network,100.0,2011,web,52cdef7c4bab8bd67529831a,17100000.0,Los Angeles,USA,37.446823,-122.161523,"{'type': 'Point', 'coordinates': [-122.161523,...",3
3,5e049e9de6eb90cad2d707b0,Skydeck,9.0,2012,web,52cdef7c4bab8bd675298509,4000000.0,Palo Alto,USA,37.564538,-122.325470,"{'type': 'Point', 'coordinates': [-122.3254705...",2
4,5e049e9de6eb90cad2d707b1,PeekYou,20.0,2012,web,52cdef7c4bab8bd675297f94,1830000.0,New York,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",6
9,5e049e9de6eb90cad2d707b7,Inuvo,50.0,2009,web,52cdef7d4bab8bd67529980b,4200000.0,Clearwater,USA,27.913011,-82.702449,"{'type': 'Point', 'coordinates': [-82.702449, ...",2


In [10]:
geo = []
for coords in data_companies['coordinates']:
    geo.append(coords['coordinates'])

In [12]:
 # Latitud 37
# Longitud -122

In [15]:
def getNearby(lat,lng,query,radius):
        CLIENT_ID = os.getenv("client_id")
        CLIENT_SECRET = os.getenv("client_secret")
        url = 'https://api.foursquare.com/v2/venues/explore'
        queryParams = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v='20181023', # DONDE ESTOY
            ll=f"{lat},{lng}", #QUE QUIERO
            query=query, # A QUE DISTANCIA
            radius=radius
        )
        res = requests.get(url, params=queryParams)
        return res.json()
  

In [63]:
# Lo comento porque al hacer Run, no quiero volver a hacer las requests
"""p = []
for coord in geo:
    for place, value in places.items():
        try:
            request = (getNearby(coord[1],coord[0],value['api'],value['distance']))
            p.append(request)
        except:
            Exception: f"Ha habido problemas con {place}"
p"""

[{'meta': {'code': 200, 'requestId': '5e0738eb963d29001bdf5d0a'},
  'response': {'suggestedFilters': {'header': 'Tap to show:',
    'filters': [{'name': '$-$$$$', 'key': 'price'}]},
   'headerLocation': 'Downtown San Mateo',
   'headerFullLocation': 'Downtown San Mateo, San Mateo',
   'headerLocationGranularity': 'neighborhood',
   'query': 'starbucks coffee',
   'totalResults': 43,
   'suggestedBounds': {'ne': {'lat': 37.57587900900001,
     'lng': -122.31256175941458},
    'sw': {'lat': 37.55787899099999, 'lng': -122.33522824058541}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recommended',
     'items': [{'reasons': {'count': 0,
        'items': [{'summary': 'This spot is popular',
          'type': 'general',
          'reasonName': 'globalInteractionReason'}]},
       'venue': {'id': '4ae07de4f964a5209a7f21e3',
        'name': 'Starbucks',
        'location': {'address': '54 E 4th Ave',
         'crossStreet': 'btwn El Camino Real & San Mateo',
         'lat': 37.5

In [94]:
p[0]['response']["groups"][0]["items"][0]['venue']['location']['city']

'San Mateo'

In [95]:
def curateResponse(data):
    places = []
    for item in data:
        for i in item["response"]["groups"][0]["items"]:
            places.append({
                "type": i["venue"]['categories'][0]['name'],
                "country": i["venue"]['location']['country'],
                #"city": i["venue"]['location']['city'],
                "name":i["venue"]['name'],
                "distance": i["venue"]['location']["distance"],
                "coordinates": {
                "type":"Point",
                "coordinates":[i["venue"]['location']["lng"], i["venue"]['location']["lat"]]
                    }})
    return places

In [96]:
df = pd.DataFrame(curateResponse(p))

In [97]:
df.type.unique()

array(['Coffee Shop', 'Airport', 'Airport Lounge', 'Airport Terminal',
       'Rental Car Location', 'Plane', 'Airport Gate', 'Music School',
       'Gym', 'Bar', 'Toy / Game Store', 'Grocery Store',
       'German Restaurant', 'Japanese Restaurant', 'Wine Bar',
       'Supermarket', 'Ramen Restaurant', 'Sushi Restaurant',
       'Burmese Restaurant', 'Taxi', 'Building', 'Airport Service',
       'Travel Agency', 'Pizza Place', 'Cajun / Creole Restaurant',
       'Chinese Restaurant', 'Mexican Restaurant', 'Sports Bar',
       'Israeli Restaurant', 'Cycle Studio', 'Italian Restaurant',
       'American Restaurant', 'Caribbean Restaurant',
       'Brazilian Restaurant', 'Nightclub', 'Bakery', 'Hotel',
       'Arts & Crafts Store', 'Transportation Service', 'Theater',
       'Jazz Club', 'Strip Club', 'Theme Restaurant', 'Plaza', 'Gay Bar',
       'Cuban Restaurant', 'New American Restaurant',
       'Performing Arts Venue', 'Lounge', 'Diner', 'Burger Joint', 'Pub',
       'Bowling Alley

In [98]:
df.type.value_counts().head()

Coffee Shop         424
Airport Terminal    306
Airport             112
Airport Gate         80
Bar                  53
Name: type, dtype: int64

In [99]:
# Vamos a organizar los datos, y solo nos quedaremos en la columna 'type' con los valores 
# del diccionario <groups> 
groups= {
    "Coffee Shop": 'Coffee Shop',
    "Airport": 'Airport',
    "Bar": ["Bar","Nightcub","Cocktail Bar","Pub","Karaoke Bar"],
    "School":[ "School","High School"]
}
def chooseCat(prod):
    for groupName, groupItems in groups.items():
        if prod in groupItems:
            return groupName
    return "OTHER"

In [100]:
df["category"] = df.type.apply(chooseCat)

In [101]:
df.category.value_counts()

OTHER          911
Coffee Shop    424
Airport        112
Bar             94
School           4
Name: category, dtype: int64

In [102]:
df = df[df.category != 'OTHER']

In [103]:
data = df.reset_index(drop=True)

In [48]:
import json
data.to_json('./queries.json', orient='records',default_handler=str)

In [105]:
# Voy a intentar unir los dos DF: el de las compañias y el de las quieries:
result = pd.concat([data_companies, data])

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [107]:
result.country.unique()

array(['USA', 'SWE', 'GBR', 'NLD', 'United States', 'Sverige',
       'United Kingdom', 'Nederland'], dtype=object)

In [109]:
result.loc[(result['country']== 'United States'), 'country'] = 'USA'
result.loc[(result['country']== 'Sverige'), 'country'] = 'SWE'
result.loc[(result['country']== 'United Kingdom'), 'country']  = 'GBR'
result.loc[(result['country']== 'Nederland'), 'country']  = 'NLD'


In [110]:
result.country.unique()

array(['USA', 'SWE', 'GBR', 'NLD'], dtype=object)

In [111]:
new_data = result[["coordinates", "country", "category", "name"]]

In [113]:
new_data.head(-10)

,coordinates,country,category,name
0,"{'type': 'Point', 'coordinates': [-122.323895,...",USA,web,Fixya
2,"{'type': 'Point', 'coordinates': [-122.161523,...",USA,web,Social Gaming Network
3,"{'type': 'Point', 'coordinates': [-122.3254705...",USA,web,Skydeck
4,"{'type': 'Point', 'coordinates': [-73.985506, ...",USA,web,PeekYou
9,"{'type': 'Point', 'coordinates': [-82.702449, ...",USA,web,Inuvo
...,...,...,...,...
619,"{'type': 'Point', 'coordinates': [-74.00172, 4...",USA,Coffee Shop,Starbucks
620,"{'type': 'Point', 'coordinates': [-74.00034, 4...",USA,Coffee Shop,Starbucks
621,"{'type': 'Point', 'coordinates': [-74.00261, 4...",USA,Coffee Shop,Starbucks
622,"{'type': 'Point', 'coordinates': [-73.999567, ...",USA,Coffee Shop,Starbucks
